In [1]:
import re
import shutil
import pandas as pd
from pandas import DataFrame, Series
import github3
from github3 import login
from git import Repo, GitCommandError
import yaml
from subprocess import Popen, PIPE, STDOUT, SubprocessError, check_output
from configparser import ConfigParser
from github3.repos.branch import Branch
from github3.null import NullObject
from schematics.models import Model, BaseType
from schematics.types import StringType, IntType, BooleanType
from schematics.types.compound import ListType, DictType
from schematics.exceptions import ValidationError
%cd ~/Documents/projects/stitch/python/
from stitch.core.stitch_frame import StitchFrame
%cd ~/Documents/projects/nerve/python/
from nerve.core.model import Nerve
from nerve.core.client import Client
from nerve.core.utils import *
from nerve.core.git import Git
from nerve.core.git_lfs import GitLFS
from nerve.core.utils import execute_subprocess, get_status
from nerve.spec.specifications import *
from nerve.spec.validators import *
from nerve.spec.traits import *
from nerve.spec import traits
from nerve.test.test_project import *
%cd ~/Documents/projects/nerve

/Users/alex/Documents/projects/stitch/python
/Users/alex/Documents/projects/nerve/python
/Users/alex/Documents/projects/nerve


In [2]:
# config_path = '/Users/alex/Documents/projects/nerve/python/nerve/resources/bad-nerverc.yml'
config_path = '/Users/alex/Documents/projects/nerve/python/nerve/resources/nerverc.yml'
# conf = yaml.load(open(config_path, 'r'))

nerve = Nerve(config_path)
name = 'nerve-test-repo-3'
proj = os.path.join(nerve['project-root'], name)
x = nerve.config
del x['project']['gitignore']
# x

In [3]:
nerve.delete(False, True)

True

In [8]:
nerve.create(notes='some note about a thing')

True

In [4]:
nerve.clone()

True

In [10]:
%%bash
cd /tmp/nerve-test-repo-3/vol001/
mkdir nerve-test-repo-3_vol001_desc_v001_-_-_-
cd nerve-test-repo-3_vol001_desc_v001_-_-_-
echo v001 0000 > nerve-test-repo-3_vol001_desc_v001_-_-_0000.png
echo v001 0001 > nerve-test-repo-3_vol001_desc_v001_-_-_0001.png
echo v001 0002 > nerve-test-repo-3_vol001_desc_v001_-_-_0002.png
cd ../../maya001
echo vasc vasc v001 > nerve-test-repo-3_maya001_vasculature.mb
echo vasc kidney v001 > nerve-test-repo-3_maya001_kidney.mb
cd ../
tree

.
├── geo001
├── maya001
│   ├── nerve-test-repo-3_maya001_kidney.mb
│   └── nerve-test-repo-3_maya001_vasculature.mb
├── proj001_meta.yml
└── vol001
    └── nerve-test-repo-3_vol001_desc_v001_-_-_-
        ├── nerve-test-repo-3_vol001_desc_v001_-_-_0000.png
        ├── nerve-test-repo-3_vol001_desc_v001_-_-_0001.png
        └── nerve-test-repo-3_vol001_desc_v001_-_-_0002.png

4 directories, 6 files


In [11]:
stat = nerve.status(
    status_asset_types=['deliverable', 'nondeliverable'],
#     status_states=['added'],
    verbosity=2
)
d = DataFrame(list(stat)).applymap(lambda x: x.data)
st = StitchFrame(d).flatten(prefix=False)
st.to_dataframe().T

,0,1,2
asset-id,bb29d0b6-3356-4348-b399-d8...,af730169-30cb-4df7-b999-e2...,05665a05-f7d1-4b4d-9085-62...
asset-name,nerve-test-repo-3_maya001_...,nerve-test-repo-3_maya001_...,nerve-test-repo-3_vol001_d...
asset-type,nondeliverable,nondeliverable,deliverable
data,None,None,None
dependencies,NaN,NaN,[]
descriptor,NaN,NaN,desc
notes,,,
project-id,None,None,None
project-name,nerve-test-repo-3,nerve-test-repo-3,nerve-test-repo-3
specification,maya001,maya001,vol001


In [12]:
nerve.publish(notes='create vol001_v001; if you can read this, it probably worked', verbosity=2)

SubprocessError: Command: "git push origin alex" returned "error: failed to push some refs to 'git@github.com:nerve-test-organization/nerve-test-repo-3.git'"

In [18]:
command = 'git push {remote} {branch}'.format(remote='origin', branch='alex')
output = Popen(command, shell=True, stdout=PIPE, stderr=PIPE)
stderr = output.stderr.read().decode('utf-8')
output = output.stdout.readlines()
output = [x.decode('utf-8').strip('\n') for x in output]

In [6]:
test_whole_project_cycle()

In [22]:
from subprocess import TimeoutExpired, CalledProcessError

def execute_subprocess(command, error_re='[eE]rror:.*'):
    '''
    Executes a given command as a subprocess and scrubs the output for errors

    Args:
        command (str): shell command to be run
        error_re (str, optional): regex used for capturing errors

    Returns:
        list: lines of output

    Raises:
        SubprocessError: stdout error as message
    '''
    timeout=100
    with Popen(command, shell=True, stdout=PIPE, stderr=PIPE) as process:
        try:
            stdout, stderr = process.communicate(input, timeout=timeout)
        except TimeoutExpired:
            process.kill()
            stdout, stderr = process.communicate()
            raise TimeoutExpired(process.args, timeout, output=stdout,
                                 stderr=stderr)
        except:
            process.kill()
            process.wait()
            raise
        retcode = process.poll()
#         if check and retcode:
#             raise CalledProcessError(retcode, process.args,
#                                      output=stdout, stderr=stderr)
    
    return retcode, stdout, stderr

#     stderr = output.stderr.read().decode('utf-8')
#     output = output.stdout.readlines()
#     output = [x.decode('utf-8').strip('\n') for x in output]

#     err = '\n'.join(output)
#     for e in [err, stderr]:
#         error = re.search(error_re, e)
#         if error:
#             message = 'Command: "' + command
#             message += '" returned "' + error.group(0) + '"'
#             raise SubprocessError(message)

#     return output

execute_subprocess('GIT_TRACE=1; git add --all; git commit -m "testmagoo"; git push origin alex')

(1,
 b'[alex 2e9b407] testmagoo\n 2 files changed, 6 insertions(+)\n create mode 100644 maya001/nerve-test-repo-3_maya001_kidney2.mb\n create mode 100644 maya001/nerve-test-repo-3_maya001_vasculature2.mb\n\rGit LFS: (0 of 2 files) 0 B / 22 B                                             \n',
 b"LFS: Git credentials for http://localhost:8080 not found.\nLFS: Git credentials for http://localhost:8080 not found.\nerror: failed to push some refs to 'git@github.com:nerve-test-organization/nerve-test-repo-3.git'\n")

In [5]:
repo = Git(proj)
repo.add(
['maya001/nerve-test-repo-3_maya001_kidney.mb', 'maya001/nerve-test-repo-3_maya001_vasculature.mb']
)
repo.commit('aekdfgjhsadgfjh')
repo.push('alex')

FileNotFoundError: [Errno 2] No such file or directory: 'git push origin alex'

In [13]:
def patch(name, nerve, user, username):
    path = '/Volumes/data/nerve-test-projects/' + user
    if not os.path.exists(path):
        os.mkdir(path)
    nerve._config['project-root'] = '/Volumes/data/nerve-test-projects/' + user
    nerve._config['user-branch'] = user
    nerve._config['username'] = username
    nerve.clone(name)

branches = [
#     ('alex', 'theNewFlesh'),
    ('nerve-test-user', 'nerve-test-user')
]
for user, username in branches:
    patch(name, nerve, user, username)

In [9]:
def create_command(branch, version, asset):
    cmd = 'echo v{v} > {a}001/ntr_{a}_v{v}.txt; git add --all; git commit -m "v{v}";'
#     cmd += ' git push origin {b}'
    cmd = cmd.format(a=asset, b=branch, v=version)
    return cmd

def do(branch, versions, asset):
    cmds = []    
    cmds.append('git checkout {b}'.format(b=branch))
    cmds.append('git pull origin dev:{b}'.format(b=branch))
    for ver in versions:
        cmds.append(create_command(branch, ver, asset))
    cmds.append('git push origin {b}'.format(b=branch))
    path = '/Users/alex/Documents/data/nerve-test-projects/' + branch + '/nerve-test-repo'
#     os.chdir(path)
    for cmd in cmds: 
        print(cmd)
#         execute_subprocess(cmd)

In [10]:
do('nerve-test-user', ['001', '002'], 'vol')
# do('nerve-test-user', ['001', '002'], 'geo')
# do('alex', ['003', '004'], 'vol')
# do('nerve-test-user', ['003', '004'], 'geo')

git checkout nerve-test-user
git pull origin dev:nerve-test-user
echo v001 > vol001/ntr_vol_v001.txt; git add --all; git commit -m "v001";
echo v002 > vol001/ntr_vol_v002.txt; git add --all; git commit -m "v002";
git push origin nerve-test-user


In [7]:
!git checkout dev; git merge nerve-test-user -m 'blah'; git push origin dev; git checkout nerve-test-user

Switched to branch 'dev'
Your branch is up-to-date with 'origin/dev'.
Updating 6900604..390a4da
Fast-forward (no commit created; -m option ignored)
 vol001/ntr_vol_v001.txt | 1 +
 vol001/ntr_vol_v002.txt | 1 +
 2 files changed, 2 insertions(+)
 create mode 100644 vol001/ntr_vol_v001.txt
 create mode 100644 vol001/ntr_vol_v002.txt
Total 0 (delta 0), reused 0 (delta 0)
To git@github.com:nerve-test-organization/nerve-test-repo.git
   6900604..390a4da  dev -> dev


In [15]:
do('alex', ['001', '002'], 'geo')

git checkout alex
git pull origin dev:alex
echo v001 > geo001/ntr_geo_v001.txt; git add --all; git commit -m "v001";
echo v002 > geo001/ntr_geo_v002.txt; git add --all; git commit -m "v002";
git push origin alex


In [10]:
!git checkout dev; git merge alex -m 'blah'; git push origin dev; git checkout alex

Switched to branch 'dev'
Your branch is behind 'origin/dev' by 2 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)
Updating 6900604..39e8196
Fast-forward (no commit created; -m option ignored)
 geo001/ntr_geo_v001.txt | 1 +
 geo001/ntr_geo_v002.txt | 1 +
 vol001/ntr_vol_v001.txt | 1 +
 vol001/ntr_vol_v002.txt | 1 +
 4 files changed, 4 insertions(+)
 create mode 100644 geo001/ntr_geo_v001.txt
 create mode 100644 geo001/ntr_geo_v002.txt
 create mode 100644 vol001/ntr_vol_v001.txt
 create mode 100644 vol001/ntr_vol_v002.txt
Total 0 (delta 0), reused 0 (delta 0)
To git@github.com:nerve-test-organization/nerve-test-repo.git
   390a4da..39e8196  dev -> dev
Switched to branch 'alex'


In [8]:
do('alex', ['003', '004'], 'vol')
do('dev', ['003', '004'], 'geo')

git checkout alex
echo v003 > vol001/ntr_vol_v003.txt; git add --all; git commit -m "v003";
echo v004 > vol001/ntr_vol_v004.txt; git add --all; git commit -m "v004";
git push origin alex
git checkout dev
echo v003 > geo001/ntr_geo_v003.txt; git add --all; git commit -m "v003";
echo v004 > geo001/ntr_geo_v004.txt; git add --all; git commit -m "v004";
git push origin dev


In [17]:
username = 'theNewFlesh'
name = 'nerve-test-repo'
token = 'e17afcc8c42fe24536c14700c4931f0e7f252bb1'
client = login(username, token=token)
orgname = 'nerve-test-organization'
org = client.organization(orgname)
repo = client.repository(orgname, name)
# {team.id: team.name for team in org.teams()}

In [22]:
uri = 'git@github.com:nerve-test-organization/nerve-test-repo.git'
path = '/Users/alex/Documents/data/nerve-test-projects/temp/nerve-test-repo'
x = Repo.clone_from(uri, path, branch='dev')